# Exploración y clasificación del Glass Dataset en Python
#### Por: Tomás Garzón Gómez, Daniel Garzón Fraile, Santiago Villota Alava 

### Exploración del Dataset

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Propiedades del Dataset

6.5


### Preparación de los datos

### Evaluación de Algoritmos

##### Algortimo \#1

##### Algoritmo #2

##### Algoritmo #3

##### Algoritmo #4

### Análisis y comparación de algoritmos

### Prueba final con algoritmo x sobre todo el set de datos